# Chain Abstractions

In [1]:
import json
import random
import tracery
from tracery.modifiers import base_english

### Load Words and Clues

In [2]:
four = json.load(open("../../corpora/data/words/word_clues/clues_four.json"))
five = json.load(open("../../corpora/data/words/word_clues/clues_five.json"))
six = json.load(open("../../corpora/data/words/word_clues/clues_six.json"))

source = dict(four["data"], **five["data"], **six["data"])

### Load Compound Words

In [3]:
compounds = json.load(open("../../corpora/data/words/compounds.json"))

When fed a compound word (e.x. foothill), this function will take the second inner word (e.x. hill) and find another that has a matching first (e.x. hilltop).

In [4]:
def findMatch(i, this_thread, compounds):
    for j in compounds["compounds"]:
        if i["secondWord"] == j["firstWord"]:
            if len(this_thread) == 0 or i != this_thread[-1]:
                this_thread.append(i)
            this_thread.append(j)
            i = j
            this_thread = findMatch(i, this_thread, compounds)
            break
            
    return this_thread

Get all possible threads in compounds.json

In [5]:
all_threads = []
this_thread = []

for i in compounds["compounds"]:
    final_thread = findMatch(i, this_thread, compounds)
    
    if len(final_thread) != 0:
        all_threads.append(final_thread)
        
    this_thread = []

### Matching Threads to Clues

1. Set a thread in all_threads to poem_thread
2. Take first word in poem_thread
3. Try to find match in clue words
4. If match, set key (firstWord, secondWord) tuple to clue list

In [6]:
poem_threads_all = []
poem_thread = {}

for thread in all_threads:
    for i in range(len(thread)):
        if thread[i]["firstWord"] in source.keys():
            this_word = (thread[i]["firstWord"],thread[i]["secondWord"])
            poem_thread[this_word] = source[thread[i]["firstWord"]]
            
        else:
            break
            
    poem_threads_all.append(poem_thread)
    poem_thread = {}

Clean up poem_threads; delete empty or 1 word threads.

In [7]:
poem_threads_len = len(poem_threads_all)
poem_threads = []

for i in range(poem_threads_len - 1):
    if len(poem_threads_all[i]) > 1:
        poem_threads.append(poem_threads_all[i])

### Get Nouns, Verbs, Adjectives, and Prepositions

Gather a dataset of nouns, verbs, adjectives, and prepositions to reference words to in hopes of finding its type.

In [8]:
import spacy
nlp = spacy.load("en_core_web_md")

- Nouns = "a/an"
- Verbs = "to"
- Adjectivess/Prepositionss = "to be"
- Not Found = ""

In [9]:
def getType(word):
    doc = nlp(word)
    
    for word in doc:
        if word.pos_ == "ADJ" or word.pos_ == "ADP":
            return "to be"

        elif word.pos_ == "VERB":
            return "to"

        elif word.pos_ == "NOUN":
            return "a"

        else:
            return ""

### Put it All Together!

Take a random poem thread, get types, and print it out.

In [10]:
poem_thread = random.choice(poem_threads)

last_phrase = ""
last_word = ""
first = True

for key,value in poem_thread.items():
    rules = {
        "body_first": "#phrase.capitalize# #word#\n    is #phrase# #clue#,\n#numSpace#but #phrase# #compound# is",
        "body_noun_first": "#word.a.capitalize#\n    is #clue.a#,\n#numSpace#but #compound.a# is",
        
        "body": "#phrase# #word#\n    is #phrase# #clue#,\n#numSpace#but #phrase# #compound# is",
        "body_noun": "#word.a#\n    is #clue.a#,\n#numSpace#but #compound.a# is",

        "phrase": getType(key[0]),
        "word": key[0],
        "clue": value,
        "numSpace": " "*(len("#clue#") + len(key[1])),
        "compound": key[0] + key[1]
    }

    grammar = tracery.Grammar(rules)
    grammar.add_modifiers(base_english)
    
    if first and getType(key[0]) == "a":
        print(grammar.flatten("#body_noun_first#"))
        first = False
        
    elif first:
        print(grammar.flatten("#body_first#"))
        first = False
        
    elif getType(key[0]) == "a":
        print(grammar.flatten("#body_noun#"))
        
    else:
        print(grammar.flatten("#body#"))
    
    last_phrase = getType(key[0])
    last_word = key[1]


last_line_rules = {
    "last": "#phrase# #word#?",
    "last_noun": "#word.a#?",
    "phrase": last_phrase,
    "word": last_word
}
grammar = tracery.Grammar(last_line_rules)
grammar.add_modifiers(base_english)

if last_phrase == "a":
    print(grammar.flatten("#last_noun#"))

else:
    print(grammar.flatten("#last#"))

To be down
    is to be not up,
            but to be downstream is
a stream
    is a water source,
          but a streamline is
a line?
